# SciPy

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
try:
    from scipy.integrate import solve_ivp
except ImportError:
    print('missing solve_ivp')
    from scipy.integrate import odeint
from scipy.interpolate import interp1d
from scipy import optimize

In [ ]:
import scipy
scipy.__version__

## ODE - solve_ivp

Note: This is a conversion from the Scipy Lectures of the old ODE solver to the new one.

Consider the ODE:

$\frac{dy}{dt} = -2*y$ 

Solve if for $t = 0...4$ and $y(t=0) = 1$

In [ ]:
def dydt(t, y): 
    return -2 * y

t_span = [0, 4]
y0 = [1]

sol = solve_ivp(dydt, t_span, y0)

fig, ax = plt.subplots(figsize=(10,8))
ax.plot(sol.t, sol.y[0], label='y0=1')
ax.legend()

In [ ]:
sol

In [ ]:
def exponential_decay(t, y): 
    return -0.5 * y

t_span = [0, 10]
y0 = [2, 4, 8]
sol = solve_ivp(exponential_decay, t_span, y0)


In [ ]:
sol

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
ax.plot(sol.t, sol.y[0], label='y0=2')
ax.plot(sol.t, sol.y[1], label='y0=4')
ax.plot(sol.t, sol.y[2], label='y0=8')
ax.legend()

## ODE - odeint

This is using the old odeint code.

Consider the ODE:

$\frac{dy}{dt} = -2*y$ 

Solve if for $t = 0...4$ and $y(t=0) = 1$

In [ ]:
def dydt(y, t):  # note order of y/t different from solve_ivp
    return -2 * y

ts = np.linspace(0, 4, 100)  
y0 = 1
ys = odeint(dydt, y0, ts)
plt.plot(ts, ys)

## ODE Exercise:

* Solve the equation:

$\frac{dy}{dt} = \frac{-4}{e^{1/y}}$ 

for $t = 0...10$ and $y(t=0) = 5$

## Interpolation

https://imgs.xkcd.com/comics/curve_fitting.png

In [ ]:
xkcd_df = pd.read_csv('../data/xkcd-2048.csv')
xkcd_df.plot.scatter(x='x', y='y')

In [ ]:
xkcd = xkcd_df.values
xkcd

In [ ]:
# linear model
def lin_fn(x, m, b):
    return m*x + b

# can't have nans, infs in x/y
params, params_covariance = optimize.curve_fit(lin_fn, xkcd[:,0], xkcd[:,1])
print(params)

m, b = params
x = np.linspace(50, 180)
fig, ax = plt.subplots(figsize=(10,8))
xkcd_df.plot.scatter(x='x', y='y', ax=ax, c='k')
ax.plot(x, x*m + b, c='r', linewidth=3)

In [ ]:
# quad model
def quad_fn(x, a, b, c):
    return a*x**2 + b*x + c

# can't have nans, infs in x/y
params, params_covariance = optimize.curve_fit(quad_fn, xkcd[:,0], xkcd[:,1])
print(params)

a, b, c = params
x = np.linspace(50, 180)
fig, ax = plt.subplots(figsize=(10,8))
xkcd_df.plot.scatter(x='x', y='y', ax=ax, c='k')
_ = ax.plot(x, a*x**2 + b*x + c, c='r', linewidth=3)

In [ ]:
# log model
def log_fn(x, a, b, c):
    return a*np.log(b*x) + c

# can't have nans, infs in x/y
params, params_covariance = optimize.curve_fit(log_fn, xkcd[:,0], xkcd[:,1])
print(params)

a, b, c = params
x = np.linspace(50, 180)
fig, ax = plt.subplots(figsize=(10,8))
xkcd_df.plot.scatter(x='x', y='y', ax=ax, c='k')
_ = ax.plot(x, a*np.log(b*x) + c, c='r', linewidth=3)

In [ ]:
# sometimes the seed is important

# exp model
def exp_fn(x, a, b, c):
    return a*np.exp(b*x) + c

p0 = (1, .1, 1)
#p0 = (3, .2, 30)
# can't have nans, infs in x/y
params, params_covariance = optimize.curve_fit(exp_fn, xkcd[:,0], xkcd[:,1], p0=p0)
print(params)

a, b, c = params
x = np.linspace(50, 180)
fig, ax = plt.subplots(figsize=(10,8))
xkcd_df.plot.scatter(x='x', y='y', ax=ax, c='k')
_ = ax.plot(x, exp_fn(x, a, b, c), c='r', linewidth=3)

In [ ]:
# flat model
def flat_fn(x, a):
    return [a] * len(x)

# can't have nans, infs in x/y
params, params_covariance = optimize.curve_fit(flat_fn, xkcd[:,0], xkcd[:,1])
print(params)

a = params
x = np.linspace(50, 180)
fig, ax = plt.subplots(figsize=(10,8))
xkcd_df.plot.scatter(x='x', y='y', ax=ax, c='k')
_ = ax.plot(x, flat_fn(x, a), c='r', linewidth=3)

In [ ]:
# hacking to figure out sigmoid initial values
def sigmoid_fn3(x, a, b, c, d):
    return a / (1. + np.exp(-c * (x - d))) + b

x = np.linspace(-10, 200, 100)
args = (100, 30, .1, 100)
y = sigmoid_fn3(x, *args)
plt.plot(x, y)

In [ ]:
# log reg (sigmoid) model
def sigmoid_fn3(x, a, b, c, d):
    return a / (1. + np.exp(-c * (x - d))) + b

p0 =(100, 30, .01, 110)
#p0 = (1, 1, 1, 1)  # this gives a flat line
# p0 = (100,1,.01,1) # this works
# can't have nans, infs in x/y
params, params_covariance = optimize.curve_fit(
    sigmoid_fn3, xkcd[:,0], xkcd[:,1], 
    p0=p0
)
print(params)

x = np.linspace(50, 180)
fig, ax = plt.subplots(figsize=(10,8))
xkcd_df.plot.scatter(x='x', y='y', ax=ax, c='k')
_ = ax.plot(x, sigmoid_fn3(x, *params), c='r', linewidth=3)

In [ ]:
# overfitting with sklearn
from sklearn.tree import DecisionTreeRegressor
dtr = DecisionTreeRegressor()
dtr.fit(xkcd_df[['x']], xkcd_df['y'])

x = np.linspace(50, 180, 300)
fig, ax = plt.subplots(figsize=(10,8))
xkcd_df.plot.scatter(x='x', y='y', ax=ax, c='k')
_ = ax.plot(x, dtr.predict(x.reshape(len(x), 1)), c='r', linewidth=3)

## Lab data

Loading Anscombe's Quartet

In [ ]:
ans = pd.read_csv('../data/anscombes.csv')
g = ans.groupby('dataset')
(_, ans1),(_, ans2),(_, ans3),(_, ans4) = list(g)

## Exercise:

* Solve each or the 4 datasets of Anscombe's quartet with an equation.
* Plot the solution for each

## Curve fitting Snow Depth

In [ ]:
snow_df = pd.read_csv('../data/snow-alta-1990-2017.csv')

In [ ]:
snow_arr_y = snow_df.SNWD.interpolate().values
snow_arr_x = np.linspace(0, len(snow_arr_y), num=len(snow_arr_y))

In [ ]:
snow_df[snow_df.SNWD.isnull()]

In [ ]:
plt.plot(snow_arr_x, snow_arr_y)

In [ ]:
optimize.least_squares

In [ ]:
# linear model
def lin_fn(x, m, b):
    return m*x + b

# can't have nans, infs in x/y
params, params_covariance = optimize.curve_fit(lin_fn, snow_arr_x, snow_arr_y)
params

In [ ]:
ax, fig = plt.subplots(figsize=(10,8))
plt.plot(snow_arr_x, snow_arr_y)
m, b = params
plt.plot(snow_arr_x, snow_arr_x*m + b)

In [ ]:

# sine model
def sin_fn(x, amp, freq, phase, offset):
    return amp*np.sin(freq*x+phase) + offset

size = 2400
#size = len(snow_arr_y)

amp = 1000
freq = 3*np.std(snow_arr_y[:size])/(2**.5)
freq = 2e-2
phase = 1
offset = 1
p0 = [amp, freq, phase, offset]
# can't have nans, infs in x/y

params_sin, params_covariance = optimize.curve_fit(
    sin_fn, snow_arr_x[:size], snow_arr_y[:size], p0=p0)
print(params_sin)
ax, fig = plt.subplots(figsize=(10,8))
plt.plot(snow_arr_x[:size], snow_arr_y[:size])
a, b, c, d = params_sin
#plt.plot(snow_arr_x, a*np.sin(b*snow_arr_x) + c)
#plt.plot(snow_arr_x, sin_fn(snow_arr_x, a*400, 10e-3*b, c, d))
plt.plot(snow_arr_x[:size], sin_fn(snow_arr_x[:size], a, b, c, d))
